## Installations

In [ ]:
!pip install crewai crewai-tools langchain langchain-community langchain-openai faiss-cpu pandas beautifulsoup4

## Data Loading and Preprocessing

In [ ]:
import os
from google.colab import userdata

# Load secrets from Colab's secret manager
os.environ["AZURE_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["AZURE_API_BASE"] = userdata.get('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_API_VERSION"] = userdata.get('OPENAI_API_VERSION')
os.environ["AZURE_DEPLOYMENT_ID"] = userdata.get('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')
os.environ["AZURE_EMBEDDING_DEPLOYMENT_NAME"] = userdata.get('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME') # Add this line
os.environ["OPENAI_API_TYPE"] = 'azure' # Keep this to explicitly set the provider type for LiteLLM

os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
print("Secrets loaded successfully.")

In [ ]:
from langchain_community.document_loaders import CSVLoader

In [ ]:
def load_and_process_data():
    """Loads data from CSV files into a preliminary document format."""
    print("\n--- Step 1: Loading and Processing Data ---")
    quran_docs, hadith_docs = [], []
    try:
        quran_loader = CSVLoader(file_path='main_df.csv', encoding='utf-8')
        quran_docs = quran_loader.load()
        hadith_loader = CSVLoader(file_path='all_hadiths_clean.csv', encoding='utf-8')
        hadith_docs = hadith_loader.load()
    except FileNotFoundError as e:
        print(f"Error: {e}. Please ensure CSV files are in the same directory.")
        return None, None
    print("Data loading complete.")
    return quran_docs, hadith_docs

In [ ]:
# Prepare data and retrievers
quran_docs, hadith_docs = load_and_process_data()
if not (quran_docs and hadith_docs):
    exit("Data loading failed. Exiting.")

## Vectorized Data Storing

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
def ingest_and_vectorize(quran_docs, hadith_docs):
    """Converts documents into AI-readable vectors and stores them for fast retrieval."""
    print("\n--- Step 2: Ingesting and Vectorizing Data ---")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    quran_splits = text_splitter.split_documents(quran_docs)
    hadith_splits = text_splitter.split_documents(hadith_docs)

    try:
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.environ["AZURE_EMBEDDING_DEPLOYMENT_NAME"],
            api_key=os.environ["AZURE_API_KEY"],
            azure_endpoint=os.environ["AZURE_API_BASE"],
            api_version=os.environ["AZURE_API_VERSION"]
        )
    except KeyError as e:
         print(f"Azure environment variable not set: {e}.")
         return None, None

    print("  - Creating FAISS vector stores...")
    quran_vector_store = FAISS.from_documents(documents=quran_splits, embedding=embeddings)
    hadith_vector_store = FAISS.from_documents(documents=hadith_splits, embedding=embeddings)
    print("Vector stores created.")
    return quran_vector_store.as_retriever(k=5), hadith_vector_store.as_retriever(k=5)

In [ ]:
# Ingest and Vectorize data
quran_retriever, hadith_retriever = ingest_and_vectorize(quran_docs, hadith_docs)
if not (quran_retriever and hadith_retriever):
    exit("Vectorization failed. Exiting.")

## Define Tools

In [ ]:
from crewai_tools import BaseTool, SerperDevTool

class ReligiousTextSearchTool(BaseTool):
    name: str = "Religious Text Search Tool"
    description: str = "Searches Qur'an and Hadith vectorstores for texts relevant to a query."
    quran_retriever: object
    hadith_retriever: object

    def _run(self, query: str) -> str:
        quran_results = self.quran_retriever.invoke(query)
        hadith_results = self.hadith_retriever.invoke(query)
        context = "QURANIC SOURCES:\n" + "\n\n".join([doc.page_content for doc in quran_results])
        context += "\n\nHADITH SOURCES:\n" + "\n\n".join([doc.page_content for doc in hadith_results])
        return context

In [ ]:
# Instantiate tools
religious_search_tool = ReligiousTextSearchTool(
    quran_retriever=quran_retriever,
    hadith_retriever=hadith_retriever
)
serper_tool = SerperDevTool()

## Define Agents

In [ ]:
from crewai import Agent
from langchain_openai import AzureChatOpenAI

def define_agents(llm, religious_tool, web_tool):
    """Defines the team of AI agents with their roles, goals, and tools."""
    researcher = Agent(
        role='Primary Source Researcher',
        goal='Find foundational texts from the Qur\'an and Hadith relevant to the user\'s query on {topic}.',
        backstory='An expert in Islamic scriptures, skilled at navigating vast digital libraries of religious texts to find the most relevant passages.',
        tools=[religious_tool], llm=llm, verbose=True
    )
    validator = Agent(
        role='Contemporary Validator',
        goal='Find contemporary views, news, and fatwas on {topic} from trusted online sources.',
        backstory='A meticulous researcher who cross-references religious findings with modern-day discourse and scholarly opinions available on the web.',
        tools=[web_tool], llm=llm, verbose=True
    )
    synthesizer = Agent(
        role='Synthesis Agent',
        goal='Craft a comprehensive, balanced, and well-structured answer to the user\'s query on {topic}, integrating primary sources and contemporary views.',
        backstory='A master communicator with deep knowledge of Islamic jurisprudence, skilled at synthesizing complex information into a clear and nuanced response.',
        llm=llm, verbose=True
    )
    return researcher, validator, synthesizer

## Define Task

In [ ]:
from crewai import Task

def define_tasks(researcher, validator, synthesizer):
    """Defines the sequence of tasks for the agents to perform."""
    research_task = Task(
        description='Search for primary texts (Qur\'an and Hadith) related to the topic: {topic}.',
        expected_output='A compiled list of relevant verses and hadiths, with full text.',
        agent=researcher
    )
    validation_task = Task(
        description='Search the web for contemporary opinions, articles, and fatwas on the topic: {topic}.',
        expected_output='A summary of key findings from diverse and reliable online sources.',
        agent=validator
    )
    synthesis_task = Task(
        description=(
            'Analyze the provided primary sources and contemporary web findings. '
            'Synthesize them into a single, comprehensive answer that addresses the user\'s query on {topic}. '
            'The answer must be well-structured, citing different viewpoints where applicable. '
        ),
        expected_output='A final, curated answer that is ready to be presented to the user.',
        agent=synthesizer,
        context=[research_task, validation_task]
    )
    return [research_task, validation_task, synthesis_task]

## Assembling the Crew

In [ ]:
from crewai import Crew, Process

# Instantiate the core LLM
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_CHAT_DEPLOYMENT_NAME"],
    api_key=os.environ["AZURE_API_KEY"],
    azure_endpoint=os.environ["AZURE_API_BASE"],
    api_version=os.environ["AZURE_API_VERSION"],
    temperature=0.7
)

In [ ]:
# Define the agents and tasks by calling the factory functions
agents = define_agents(llm, religious_search_tool, serper_tool)
tasks = define_tasks(*agents)

In [ ]:
# Assemble the Crew and kickoff the process
islamic_qna_crew = Crew(
    agents=list(agents),
    tasks=tasks,
    process=Process.sequential,
    verbose=2
)

print("\n Kicking off the Crew... ")
result = islamic_qna_crew.kickoff(inputs={'topic': 'the different scholarly views on cryptocurrency'})

print("\n\n FINAL CURATED RESPONSE ")
print("="*50)
print(result)